<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a> ©2021 onwards</font></small><hr style="margin:0;background-color:silver">

**[<font size=6>🍿Netflix</font>](https://www.kaggle.com/competitions/27nov23hse-netflix/rules)**. [**Instructions**](https://colab.research.google.com/drive/1owkYjuRGkx050LQnM3b3yTzd0Dr2XbeV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [ ]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 27nov23hse-netflix       # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show                       # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 27nov23hse-netflix
Using competition: 27nov23hse-netflix
  teamId  teamName                               submissionDate       score    
--------  -------------------------------------  -------------------  -------  
11316222  T                                      2023-12-09 14:42:12  0.42670  
11372125  Chislova Alena                         2023-12-09 13:57:17  0.42670  
11319655  B                                      2023-12-10 14:59:40  0.42040  
11368728  I                                      2023-12-10 14:20:18  0.41950  
11316599  A                                      2023-12-10 14:10:35  0.41920  
11324471  Talagaeva Polina                       2023-12-07 14:02:05  0.41830  
11362088  Mayya Spirina                          2023-12-09 16:34:36  0.41560  
11343189  S                                      2023-12-06 12:01:23  0.41390  
11297037  🍿Baseline                              2022-02-

In [ ]:
%%time
%%capture
%reset -f
!cp metadata.json METADATA
!pip -q install scikit-surprise
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, os, tqdm, re, sys, cv2, skimage, plotly.express as px, surprise
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=1000, precision=2, edgeitems=20, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 10)

CPU times: user 265 ms, sys: 20.2 ms, total: 285 ms
Wall time: 10.3 s


The matrix below contains ratings that users (user id or `uid`) have given to the movies (movie ID or `mid`). It is 85% blank, but the original dataset was 99% sparse (with ~1 rating per user) [&#127910;](https://youtu.be/ImpV70uLxyw). Your task is not to fill the whole matrix, but to impute `NaN` values in `XY_Netflix.csv`, which is an equivalent representation of `XY_Netflix_matrix.csv`. This is a collaborative filtering problem, where observations and features have some implicit relation. Recall that typically we expect independent observations in a sample (unless we have a time series or video of image frames, etc.). Here our model needs to learn and leverage the dependencies from observed movie ratings. Clustering techniques might be more intuitive here. [Truncated SVD](https://scikit-learn.org/stable/modules/decomposition.html#lsa) is also popular. Supervised models can be used as well, if we can build features describing our users (which we would treat as observations). Note that we have no other information on users except what they have ranked (which might suggest the cluster they belong to, which might offer unobserved (latent) features). [&#127910;](https://youtu.be/8wLKuscyO9I) Since users relate to other users through the movies they ranked, this can also be presented as a [bipartite graph](https://en.wikipedia.org/wiki/Bipartite_graph).

In [ ]:
dfMtx = pd.read_csv('XY_Netflix_matrix.csv', low_memory=False); dfMtx

,1,2,3,4,5,6,7,8,9,10,...,371,372,373,374,375,376,377,378,379,380
0,NaN,NaN,2.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.0,NaN,5.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,3.0,NaN,4.0,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,NaN,NaN,2.0,NaN,NaN,5.0,NaN,NaN,4.0,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128695,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
f = px.imshow(dfMtx.iloc[:2000,:].T, color_continuous_scale='RdBu_r', aspect='auto');  # movies by users (transposed for visual convenience)
f.update_layout(margin=dict(l=0,r=0,b=0,t=0))

In [ ]:
dfMtx.info()   # note the datatypes and any missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128699 entries, 0 to 128698
Columns: 380 entries, 1 to 380
dtypes: float64(380)
memory usage: 373.1 MB


The dataframe below is another representation of the matrix form above. It lists the `uid`$\times$`mid`locations of the values you need to predict (those with `NaN`) and all other non-zero values in the matrix above.

In [ ]:
df = pd.read_csv('XY_Netflix.csv', low_memory=False); df  # user ID, movie ID, rating. NaN=requires predictions

,uid,mid,rtg
0,1,1,NaN
1,2,2,NaN
2,3,3,NaN
3,4,4,NaN
4,5,5,NaN
...,...,...,...
7402067,112398,303,4.0
7402068,12926,181,4.0
7402069,122712,56,4.0
7402070,49692,126,4.0


In [ ]:
df.rtg.fillna(0).astype(int).value_counts().to_frame().T   # rating frequencies. 0 represents NaN

,4,3,5,2,1,0
rtg,2451087,2120585,1628122,842056,340222,20000


In [ ]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

**Student's Section** (between ⏳ symbols): add your code and documentation here.

## **Task 1. Preprocessing Pipeline**

Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.
1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

**Student's answer:**
First of all, we need to fulfill the missing values in order to perform a model training. For that, column_mode, row_mode  combine_imputations   mask_ratings functions were created, which then would be filled with mean amd median values of rows and columns combined. We also have a function calculating the mean absolute error of predicted matrix.

## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

 **Student's answer:**
  We performed  Singular Value Decomposition  with 2 components on the masked matrix for 70 iterations, that allowed us to do it on time without loss of accuracy. We had a mask that took true values for non-null values of the matrix. Then, we get the tall matrix with integer results, and then set their values to be appropriate(1,2,3,4,5)

# **References:**

1. https://www.kaggle.com/code/ilialar/hyperparameters-tunning-with-hyperopt#Hyperopt (https://colab.research.google.com/corgiredirector?site=https%3A%2F%2Fwww.kaggle.com%2Fcode%2Filialar%2Fhyperparameters-tunning-with-hyperopt%23Hyperopt)
1. James, G., Witten, D., Hastie, T., Tibshirani, R., & Taylor, J. (2023). An introduction to statistical learning: With applications in Python. Springer.
1. Hastie, T. (2011). The elements of statistical learning, Hastie, 2nd edition. Springer.
1. https://en.wikipedia.org/wiki/Hyperparameter_optimization#Bayesian_optimization (https://colab.research.google.com/corgiredirector?site=https%3A%2F%2Fen.wikipedia.org%2Fwiki%2FHyperparameter_optimization%23Bayesian_optimization)

In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds
from scipy import stats
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Function Definitions
def column_mode(df):
    return df.apply(lambda x: stats.mode(x)[0])

def row_mode(df):
    return df.apply(lambda x: stats.mode(x)[0], axis=1)

def combine_imputations(df, imputations):
    combined = np.nanmean(np.array(imputations), axis=0)
    return pd.DataFrame(combined, index=df.index, columns=df.columns)

def mask_ratings(df, mask_fraction=0.1):
    # Create a random mask where True indicates the values to keep
    keep_mask = np.random.rand(*df.shape) >= mask_fraction
    # Ensure that only existing ratings (non-NaN values) are considered
    mask = keep_mask | df.isna()

    # Invert the mask for masking: True for values to be set as NaN
    mask_for_nan = ~mask
    df_masked = df.mask(mask_for_nan)
    return df_masked, mask_for_nan

def calculate_mae(predictions_matrix, actuals_matrix, mask_matrix):
    # Flatten the matrices
    flat_predictions = predictions_matrix.to_numpy().flatten()
    flat_actuals = actuals_matrix.to_numpy().flatten()
    flat_mask = mask_matrix.to_numpy().flatten()

    # Filter the values where the mask is True
    filtered_predictions = flat_predictions[flat_mask]
    filtered_actuals = flat_actuals[flat_mask]

    # Calculate MAE
    mae = mean_absolute_error(filtered_predictions, filtered_actuals)
    return mae

# Data Preparation
if 'dfMtx' in vars():
    del dfMtx  # Release memory

dfK = df[df.rtg!=df.rtg]  # Kaggle submission data
dfX = df[df.uid.isin(dfK.uid.unique())].pivot_table(index='uid', columns='mid', values='rtg')
# print(myGS(dfX))
dfX_cp = dfX
tmr.ShowTime()

Runtime is 2 sec


In [ ]:
dfX = dfX_cp
#Cool validation addition
# dfX, mask_val = mask_ratings(dfX, 0.25)

# Imputation
col_mean = dfX.mean()
col_median = dfX.median()
# col_mode = column_mode(dfX)  # Optional: Include if mode computation is feasible
row_mean = dfX.mean(axis=1)
row_median = dfX.median(axis=1)
# row_mode = row_mode(dfX)  # Optional: Include if mode computation is feasible

dfX_col_mean = dfX.fillna(col_mean)
dfX_col_median = dfX.fillna(col_median)
# dfX_col_mode = dfX.fillna(col_mode)  # Optional
dfX_row_mean = dfX.apply(lambda x: x.fillna(x.mean()), axis=1)


# dfX_row_median = dfX.apply(lambda x: x.fillna(x.median()), axis=1) # Takes a long time to exec optional
# dfX_row_mode = dfX.T.fillna(row_mode).T  # Optional

# Combined Imputation
# dfX_combined = combine_imputations(dfX, [dfX_col_mean, dfX_col_median, dfX_row_mean, dfX_row_median])
dfX_combined = combine_imputations(dfX, [dfX_col_mean, dfX_col_median, dfX_row_mean])
tmr.ShowTime()

In [ ]:
# Matrix Factorization via SVD
np.random.seed(0)
n, p = dfX.shape
for _ in range(70):
    U, D, Vt = svds(dfX_combined.to_numpy(), k=2) # 30 and 15 components is too much
    dfX_combined.loc[1:n, 1:p] = U @ np.diag(D) @ Vt
    mask = ~np.isnan(dfX)
    dfX_combined[mask] = dfX[mask]
tmr.ShowTime()

In [ ]:
# print(calculate_mae(dfX_combined, dfX_cp, mask_val))
# Submissions are scored on accuracy, not on continuous/regression error
dfX_est_L = dfX_combined.unstack().reset_index().round(0).astype(int).rename(columns={0:'rtg'})  # tall format with integer ratings
dfK = dfX_est_L.merge(dfK, on=['mid','uid'], suffixes=['','_']).drop('rtg_', axis=1)
dfK.index = (dfK.uid-1)*dfK.mid.max()+dfK.mid
dfK['rtg'] = dfK['rtg'].clip(1, 5)  # restrict the range to 1,2,3,4,5 . Thanks to Yannis & Liz, See Piazza @67_f1
# Export to CSV
ToCSV(dfK[['rtg']].sort_index(), 'toKag_1')

tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.
'''0.7477501175159242 '''

In [ ]:
# def my_alg(dfX,c_param, s, cyc):
#     dfX_cp = dfX
#     mae_scores = []
#     for c in range(cyc):
#         dfX = dfX_cp
#             #Cool validation addition
#         dfX, mask_val = mask_ratings(dfX, 0.25)


#         # Imputation
#         col_mean = dfX.mean()
#         col_median = dfX.median()
#         # col_mode = column_mode(dfX)  # Optional: Include if mode computation is feasible
#         row_mean = dfX.mean(axis=1)
#         row_median = dfX.median(axis=1)
#         # row_mode = row_mode(dfX)  # Optional: Include if mode computation is feasible

#         dfX_col_mean = dfX.fillna(col_mean)
#         dfX_col_median = dfX.fillna(col_median)
#         # dfX_col_mode = dfX.fillna(col_mode)  # Optional
#         dfX_row_mean = dfX.apply(lambda x: x.fillna(x.mean()), axis=1)
#         dfX_row_median = dfX.apply(lambda x: x.fillna(x.median()), axis=1)
#         # dfX_row_mode = dfX.T.fillna(row_mode).T  # Optional

#         # Combined Imputation
#         dfX_combined = combine_imputations(dfX, [dfX_col_mean, dfX_col_median, dfX_row_mean, dfX_row_median])
#         # dfX_combined = combine_imputations(dfX, [dfX_col_mean])
#         # Matrix Factorization via SVD
#         # np.random.seed(0)
#         n, p = dfX.shape
#         for _ in range(c_param):
#             U, D, Vt = svds(dfX_combined.to_numpy(), k=s)
#             dfX_combined.loc[1:n, 1:p] = U @ np.diag(D) @ Vt
#             mask = ~np.isnan(dfX)
#             dfX_combined[mask] = dfX[mask]
#         mae = calculate_mae(dfX_combined, dfX_cp, mask_val)
#         mae_scores.append(mae)






#     # # Calculate the distribution of actual and predicted ratings
#     # actual_ratings_distribution = df['rtg'].dropna(axis=0).astype(int).value_counts().sort_index()
#     # predicted_ratings_distribution = dfK['rtg'].astype(int).value_counts().sort_index()

#     # # Normalize the distributions to get the relative frequencies
#     # total_actual = actual_ratings_distribution.sum()
#     # total_predicted = predicted_ratings_distribution.sum()

#     # actual_ratings_relative_frequency = actual_ratings_distribution / total_actual
#     # predicted_ratings_relative_frequency = predicted_ratings_distribution / total_predicted

#     # # Create a DataFrame for the proportions
#     # proportions_df = pd.DataFrame({
#     #     'Actual Proportion': actual_ratings_relative_frequency,
#     #     'Predicted Proportion': round(predicted_ratings_relative_frequency, 3)
#     # })

#     # # Display the table with proportions
#     # proportions_df.head(5)
#     # proportions_df['res'] = proportions_df['Actual Proportion'] - proportions_df['Predicted Proportion']
#     # proportions_df
#     # print(sum(abs(proportions_df.res)))
#     return np.mean(mae_scores)
# # print(my_alg(dfX))

In [ ]:
# cycles = [10]
# svd_param =[1,2,3,4,5,6,7,8,9,10,11, 12,13,14,15, 16,17, 18,19, 20]

# res = []
# for s in svd_param:
#   for c in cycles:
#     r = my_alg(dfX, c, s, cyc=5)
#     print(r)
#     res.append(str(s) + ' ' + str(c)  + ' ' + str(r))
#     print(res)

# print(res)

# '''0.7728466914126074''' #baseline
# '''0.770857353330525''' #with megaimputer

In [ ]:
# plt.subplot(1, 1, 1)
# _ = df.rtg.dropna(axis=0).astype(int).hist(density=True, alpha=0.3, color='green');
# _ = dfK.rtg.astype(int).hist(density=True, alpha=0.3, color='red');
# _ = plt.legend(['actual','predicted']);


# import pandas as pd

# # Calculate the distribution of actual and predicted ratings
# actual_ratings_distribution = df['rtg'].dropna(axis=0).astype(int).value_counts().sort_index()
# predicted_ratings_distribution = dfK['rtg'].astype(int).value_counts().sort_index()

# # Normalize the distributions to get the relative frequencies
# total_actual = actual_ratings_distribution.sum()
# total_predicted = predicted_ratings_distribution.sum()

# actual_ratings_relative_frequency = actual_ratings_distribution / total_actual
# predicted_ratings_relative_frequency = predicted_ratings_distribution / total_predicted

# # Create a DataFrame for the proportions
# proportions_df = pd.DataFrame({
#     'Actual Proportion': actual_ratings_relative_frequency,
#     'Predicted Proportion': round(predicted_ratings_relative_frequency, 3)
# })

# # Display the table with proportions
# proportions_df.head(5)
# proportions_df['res'] = proportions_df['Actual Proportion'] - proportions_df['Predicted Proportion']
# proportions_df
# print(sum(abs(proportions_df.res)))

In [ ]:
# import pandas as pd

# def calculate_dissimilarities(file1, file2):
#     # Read the CSV files
#     df1 = pd.read_csv(file1)
#     df2 = pd.read_csv(file2)

#     # Ensuring the data is aligned
#     # Assuming the predictions are aligned by index; modify as needed for your specific CSV format
#     if not df1.index.equals(df2.index):
#         raise ValueError("Indices of the two dataframes do not match.")

#     # Calculating dissimilarities
#     dissimilarities = (df1 != df2).sum().sum()

#     return dissimilarities

# # Example usage
# dissimilarity_count = calculate_dissimilarities('razeb.csv', 'toKag_1.csv')
# print(f"Number of dissimilarities: {dissimilarity_count}")

In [ ]:
# plt.subplot(1, 1, 1)
# _ = df.rtg.dropna(axis=0).astype(int).hist(density=True, alpha=0.3, color='green');
# _ = dfK.rtg.astype(int).hist(density=True, alpha=0.3, color='red');
# _ = plt.legend(['actual','predicted']);

# ToCSV(dfK[['rtg']].sort_index(), 'stop')
# dfK.shape

# # Print the distribution as numbers for actual ratings
# print("Actual Ratings Distribution:")
# actual_ratings_distribution = df['rtg'].dropna().astype(int).value_counts().sort_index()
# print(actual_ratings_distribution)

# # Print the distribution as numbers for predicted ratings
# print("\nPredicted Ratings Distribution:")
# predicted_ratings_distribution = dfK['rtg'].astype(int).value_counts().sort_index()
# print(predicted_ratings_distribution)

# # Output the shape of the submission DataFrame
# print("\ndfK Shape:", dfK.shape)


# import pandas as pd

# # Calculate the distribution of actual and predicted ratings
# actual_ratings_distribution = df['rtg'].dropna(axis=0).astype(int).value_counts().sort_index()
# predicted_ratings_distribution = dfK['rtg'].astype(int).value_counts().sort_index()

# # Normalize the distributions to get the relative frequencies
# total_actual = actual_ratings_distribution.sum()
# total_predicted = predicted_ratings_distribution.sum()

# actual_ratings_relative_frequency = actual_ratings_distribution / total_actual
# predicted_ratings_relative_frequency = predicted_ratings_distribution / total_predicted

# # Create a DataFrame for the proportions
# proportions_df = pd.DataFrame({
#     'Actual Proportion': actual_ratings_relative_frequency,
#     'Predicted Proportion': round(predicted_ratings_relative_frequency, 3)
# })

# # Display the table with proportions
# proportions_df.head(5)
# proportions_df['res'] = proportions_df['Actual Proportion'] - proportions_df['Predicted Proportion']
# proportions_df
# print(sum(proportions_df.res))

In [ ]:
# import matplotlib.pyplot as plt
# import pandas as pd

# # Calculate the distribution of actual and predicted ratings
# actual_ratings_distribution = df['rtg'].dropna(axis=0).astype(int).value_counts().sort_index()
# predicted_ratings_distribution = dfK['rtg'].astype(int).value_counts().sort_index()

# # Plot the actual ratings distribution
# plt.figure(figsize=(10, 5))
# plt.subplot(1, 2, 1)
# actual_ratings_distribution.plot(kind='bar', alpha=0.7, color='green')
# plt.title('Actual Ratings Distribution')
# plt.xlabel('Ratings')
# plt.ylabel('Count')
# plt.xticks(rotation=0)

# # Plot the predicted ratings distribution
# plt.subplot(1, 2, 2)
# predicted_ratings_distribution.plot(kind='bar', alpha=0.7, color='red')
# plt.title('Predicted Ratings Distribution')
# plt.xlabel('Ratings')
# plt.xticks(rotation=0)

# plt.tight_layout()
# plt.show()


# # Calculate the distribution of actual and predicted ratings
# actual_ratings_distribution = df['rtg'].dropna(axis=0).astype(int).value_counts().sort_index()
# predicted_ratings_distribution = dfK['rtg'].astype(int).value_counts().sort_index()

# # Normalize the distributions to get the relative frequencies
# total_actual = actual_ratings_distribution.sum()
# total_predicted = predicted_ratings_distribution.sum()

# actual_ratings_relative_frequency = actual_ratings_distribution / total_actual
# predicted_ratings_relative_frequency = predicted_ratings_distribution / total_predicted

# # Plot the actual ratings distribution (relative frequency)
# plt.figure(figsize=(12, 5))
# plt.subplot(1, 2, 1)
# actual_ratings_relative_frequency.plot(kind='bar', alpha=0.7, color='green')
# plt.title('Actual Ratings Distribution (Relative Frequency)')
# plt.xlabel('Ratings')
# plt.ylabel('Proportion')
# plt.xticks(rotation=0)

# # Plot the predicted ratings distribution (relative frequency)
# plt.subplot(1, 2, 2)
# predicted_ratings_relative_frequency.plot(kind='bar', alpha=0.7, color='red')
# plt.title('Predicted Ratings Distribution (Relative Frequency)')
# plt.xlabel('Ratings')
# plt.xticks(rotation=0)

# plt.tight_layout()
# plt.show()

In [ ]:
# import pandas as pd

# # Calculate the distribution of actual and predicted ratings
# actual_ratings_distribution = df['rtg'].dropna(axis=0).astype(int).value_counts().sort_index()
# predicted_ratings_distribution = dfK['rtg'].astype(int).value_counts().sort_index()

# # Normalize the distributions to get the relative frequencies
# total_actual = actual_ratings_distribution.sum()
# total_predicted = predicted_ratings_distribution.sum()

# actual_ratings_relative_frequency = actual_ratings_distribution / total_actual
# predicted_ratings_relative_frequency = predicted_ratings_distribution / total_predicted

# # Create a DataFrame for the proportions
# proportions_df = pd.DataFrame({
#     'Actual Proportion': actual_ratings_relative_frequency,
#     'Predicted Proportion': predicted_ratings_relative_frequency
# })

# # Display the table with proportions
# proportions_df.head(5)
# proportions_df['res'] = proportions_df['Actual Proportion'] - proportions_df['Predicted Proportion']
# proportions_df

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.sparse.linalg import svds

# # Assuming dfX_combined is your imputed matrix

# def calculate_elbow(df, max_k):
#     singular_values = []
#     for k in range(1, max_k+1):
#         U, s, Vt = svds(df, k=k)
#         singular_values.append(np.sum(s**2))

#     return singular_values

# # Define the range of k values to check
# max_k = 60  # You can adjust this value based on your dataset

# # Calculate the elbow
# elbow_values = calculate_elbow(dfX_combined.to_numpy(), max_k)

# # Plot the elbow graph
# plt.figure(figsize=(10, 6))
# plt.plot(range(1, max_k+1), elbow_values, marker='o')
# plt.xlabel('Number of Factors (k)')
# plt.ylabel('Sum of Squared Singular Values')
# plt.title('Elbow Method to Determine Optimal k')
# plt.grid(True)
# plt.show()

In [ ]:
# for a in range(1, 30):
#   dfX = dfX_cp
#   #Cool validation addition
#   dfX, mask_val = mask_ratings(dfX, 0.25)

#   # Imputation
#   col_mean = dfX.mean()
#   col_median = dfX.median()
#   # col_mode = column_mode(dfX)  # Optional: Include if mode computation is feasible
#   row_mean = dfX.mean(axis=1)
#   row_median = dfX.median(axis=1)
#   # row_mode = row_mode(dfX)  # Optional: Include if mode computation is feasible

#   dfX_col_mean = dfX.fillna(col_mean)
#   dfX_col_median = dfX.fillna(col_median)
#   # dfX_col_mode = dfX.fillna(col_mode)  # Optional
#   dfX_row_mean = dfX.apply(lambda x: x.fillna(x.mean()), axis=1)
#   dfX_row_median = dfX.apply(lambda x: x.fillna(x.median()), axis=1)
#   # dfX_row_mode = dfX.T.fillna(row_mode).T  # Optional

#   # Combined Imputation
#   dfX_combined = combine_imputations(dfX, [dfX_col_mean, dfX_col_median, dfX_row_mean, dfX_row_median])
#   # dfX_combined = combine_imputations(dfX, [dfX_col_mean])
#   # Matrix Factorization via SVD
#   np.random.seed(0)
#   n, p = dfX.shape
#   for _ in range(20):
#       U, D, Vt = svds(dfX_combined.to_numpy(), k=a)
#       dfX_combined.loc[1:n, 1:p] = U @ np.diag(D) @ Vt
#       mask = ~np.isnan(dfX)
#       dfX_combined[mask] = dfX[mask]

#   plt.subplot(1, 1, 1)
#   _ = df.rtg.dropna(axis=0).astype(int).hist(density=True, alpha=0.3, color='green');
#   _ = dfK.rtg.astype(int).hist(density=True, alpha=0.3, color='red');
#   _ = plt.legend(['actual','predicted']);


#   print(calculate_mae(dfX_combined, dfX_cp, mask_val))
#   # Submissions are scored on accuracy, not on continuous/regression error
#   dfX_est_L = dfX_combined.unstack().reset_index().round(0).astype(int).rename(columns={0:'rtg'})  # tall format with integer ratings
#   dfK = dfX_est_L.merge(dfK, on=['mid','uid'], suffixes=['','_']).drop('rtg_', axis=1)
#   dfK.index = (dfK.uid-1)*dfK.mid.max()+dfK.mid
#   dfK['rtg'] = dfK['rtg'].clip(1, 5)  # restrict the range to 1,2,3,4,5 . Thanks to Yannis & Liz, See Piazza @67_f1
#   # Export to CSV
#   ToCSV(dfK[['rtg']].sort_index(), 'toKag_1')

#   tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

#   # Calculate the distribution of actual and predicted ratings
#   actual_ratings_distribution = df['rtg'].dropna(axis=0).astype(int).value_counts().sort_index()
#   predicted_ratings_distribution = dfK['rtg'].astype(int).value_counts().sort_index()

#   # Normalize the distributions to get the relative frequencies
#   total_actual = actual_ratings_distribution.sum()
#   total_predicted = predicted_ratings_distribution.sum()

#   actual_ratings_relative_frequency = actual_ratings_distribution / total_actual
#   predicted_ratings_relative_frequency = predicted_ratings_distribution / total_predicted

#   # Create a DataFrame for the proportions
#   proportions_df = pd.DataFrame({
#       'Actual Proportion': actual_ratings_relative_frequency,
#       'Predicted Proportion': round(predicted_ratings_relative_frequency, 3)
#   })

#   # Display the table with proportions
#   proportions_df.head(5)
#   proportions_df['res'] = proportions_df['Actual Proportion'] - proportions_df['Predicted Proportion']
#   proportions_df
#   print(sum(abs(proportions_df.res)))

<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

## 💡**Starter Ideas**